# Pre-requisites

Before we get started, first make sure to install all the required tools. We provide two lists below, one needed for setting up the testbed. And one for developing code to use with the testbed. Feel free to skip the installation of the second list, and return at a later point in time.


Make sure to install a recent version of each of the dependencies.


 * (Windows only) It is strongly recommended to install every dependency in a Windows Subsystem for Linux shell. For installation refer to [here](https://docs.microsoft.com/en-us/windows/wsl/install).
 * GCloud SDK
    - Follow the installation instructions [here](https://cloud.google.com/sdk/docs/install)
    - Initialize the SDK with `gcloud init`, if prompted you may ignore to set/create a default/first project.
    - ⚠️ Run the command `gcloud auth application-default login`
        - ℹ️ We need to run this command to utilize your login credentials programmatically with terraform. This is needed as we will use these to impersonate a service account during the creation and setup of the Kubernetes cluster.
    - ⚠️ Run the command `gcloud components install beta`
        - ℹ️ We need to run this command to list the billing account IDs and enable billing. Currently, these features fall under beta access.
 * Kubectl
 * Helm
 * Terraform
 * Python3.9/10
   * jupyter, ipython, bash_kernel
```bash
pip3 install jupyter ipython bash_kernel
python3 -m bash_kernel.install
```

For development, the following tools are needed/recommended:

 * Docker (>= 18.09).
    - If you don't have experience with using Docker, we recommend following [this](https://docs.docker.com/get-started/) tutorial.
 * Python3.9
 * pip3
 * JetBrains PyCharm


# Preparation

To make sure we can request resources on Google Cloud Platform (GCP), perform the following;

1. Create a GCP account on [https://cloud.google.com](https://cloud.google.com), using a Google account
2. Redeem your academic coupon on GCP, see Brightspace for information on obtaining the \\$50 academic coupon, or use the free \\$300 credits for new users provided by Google.


3. (Non unix systems) Make sure to use the `Bash` kernel, not a Python or other kernel. For those on windows machines, make sure to launch the `jupyter notebook` server from a bash-compliant commandline, we recommend Windows Subsystem for Linux.

⚠️ Make sure to run this Notebook within a cloned repository, not standalone/downloaded from Github.

# Deployment

⚠️ This notebook assumes that commands are executed in order. Executing the provided commands multiple times should not result in issues. However, re-running cells with `cd` commands, or altering cells (other than variables as instructed) may result in unexpected behaviour.

## Getting started

First, we will set a few variables used **throughout** the project. We set them in this notebook for convenience, but they are also set as defaults in configuration files for the project. If you change any of these, make sure to change the corresponding variables as well in;

* [`terraform-gke/variables.tf`](terraform-gke/variables.tf)
* [`terraform-dependencies/variables.tf`](terraform-dependencies/variables.tf)



In [ ]:
ACCOUNT_ID="terraform-iam-service-account"
PROJECT_ID="test-bed-fltk"
PRIVILEGED_ACCOUNT_ID="${ACCOUNT_ID}@${PROJECT_ID}.iam.gserviceaccount.com"
CLUSTER_NAME="fltk-testbed-cluster"
REGION="us-central1-c"

## Project creation

Next, we create a project using the `PROJECT_ID` variable, and get all the billing account information.

In [ ]:
gcloud projects create $PROJECT_ID --set-as-default
gcloud beta billing accounts list # Copy the Account ID of the account

Copy the billing account identifier, e.g. `015594-41687F-092941`, and assign to the variable in the cell below

In [ ]:
BILLING_ACCOUNT="015594-41687F-092941"

Setup billing and enable services, this will allow us to create a GKE cluster (Google managed Kubernetes cluster), and push and pull containers to our private container repo.

In [ ]:
# Setup billing to project
gcloud beta billing projects link $PROJECT_ID --billing-account $BILLING_ACCOUNT
# Enable services now billing is enabled
gcloud services enable compute container --project $PROJECT_ID

## Creating a service-account

Create service account that has the minimum set of permissions for creating and managing a cluster. This service account
will be used to create the cluster, and deploy the dependencies that we use.

During the deployment we will make use of impersonation, to let *your* account utilize the service-account. For more information about this practise, see also [this](https://cloud.google.com/blog/topics/developers-practitioners/using-google-cloud-service-account-impersonation-your-terraform-code) blog by Google.

In [ ]:
# Helper function to quickly enable gcp roles, assumes $PRIVILEGED_ACCOUNT_ID and $PROJECT_ID to be set.
function enable_gcp_role () {
  ROLE=$1
  gcloud projects add-iam-policy-binding \
    $PROJECT_ID \
    --member="serviceAccount:$PRIVILEGED_ACCOUNT_ID" \
    --role="roles/$ROLE"
}

# Create service-account
gcloud iam service-accounts create $ACCOUNT_ID --display-name="Terraform service account" --project ${PROJECT_ID}

# Allow the service account to use the the set of roles below.
enable_gcp_role "compute.viewer"                # Allow the service account to see active resources
enable_gcp_role "storage.objectViewer"          # Allow the service account/managed resources to pull from gcr.io (your code)
enable_gcp_role "compute.networkAdmin"          # Needed for setting up private network
enable_gcp_role "compute.securityAdmin"         # Needed for GKE
enable_gcp_role "container.clusterViewer"       # Needed for GKE
enable_gcp_role "container.clusterAdmin"        # Needed for GKE
enable_gcp_role "container.developer"           # Needed for GKE
enable_gcp_role "iam.serviceAccountAdmin"       # Needed for GKE
enable_gcp_role "iam.serviceAccountUser"        # Needed for GKE


## Enable impersonation
With the service account created, we must enable impersonation, to allow the main account of the project to make use of the service account. For more information see also the [`add-iam-policy-binding`](https://cloud.google.com/sdk/gcloud/reference/iam/service-accounts/add-iam-policy-binding) reference.

Assign your `google_account` mail to the `OWNER_MAIL` variable, and run the command box below.

In [ ]:
OWNER_MAIL="jargsnork@gmail.com"
gcloud iam service-accounts add-iam-policy-binding $PRIVILEGED_ACCOUNT_ID \
 --member="user:$OWNER_MAIL" \
 --role=roles/iam.serviceAccountTokenCreator \
 --project $PROJECT_ID

To enable using your account's credentials, run the command below. This will open in a new tab/open the link that is displayed. Afterwards you can use your own credentials to impersonate the service account. 

You can, for example, also allow other google users (such as project members) to work with your cluster in this way.

In [ ]:
gcloud auth application-default login

## Creating a Google managed cluster (GKE)
To create the cluster, first change the active directory to the `terraform-gke` directory.

In [ ]:
cd ../terraform/terraform-gke

Init the directory, to initialize the Terraform module.

In [ ]:
terraform init -reconfigure

Next, we can check whether we can create a cluster. No warnings or errors should occur during this process. It may take a while to complete.

In [ ]:
terraform plan

When the previous command completes successfully, we can start the deployment. Depending on any changes you may have done, this might take a while.

By default, this will create a private zonal cluster consisting of two node-pools.

⚠️ Any changes to create a regional cluster, an additional free of \\$ 0.10 /hour will be billed with minute increments. However, only the **first** zonal cluster is free of this cost.

⚠️ By default spot/preemtible nodes are disabled, as such no discounts will be given for the deployment. You can experiment by setting `spot` to true in the `tf` files. Note, however, that the default implementations provided in the test-bed do not allow for recovering from getting rescheduled.


In [ ]:
terraform apply -auto-approve

Next, we add cluster credentials (so you can interact with the cluster through `kubectl` an `helm`).

In [ ]:
# Add credentials for interacting with cluster via kubectl
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

### Changing deployment

To save cost, or run different experiments, you might want to change the configuration of your cluster. This can be achieved by modifying the cluster configuration in the [`terraform-gke/main.tf`](../terraform/terraform-gke/main.tf) configuration file. You can change the default node-pools, create additional node pools with taints (to allow for scheduling on specific nodes/pools) and much more.

After finishing your changes, simply run the following commands

```bash
# Use `plan` to check your configuration
terraform plan
# Check to see if your changes are as expected, terraform will show what will be created/removed.

# If the changes are as you expect, apply the changes.
terraform apply #-auto-approve
```

Depending on the number of changes, this may take some time.

## Installing dependencies
Lastly, we need to install the dependencies on our cluster. First change the directories, and then run the `init`, `plan` and `apply` commands as we did for creating the GKE cluster.

In [ ]:
cd ../terraform-dependencies

Init the directory, to initialize the Terraform module.

In [ ]:
terraform init -reconfigure

Check to see if we can plan the deployment. This will setup the following:

* Kubeflow training operator (used to deploy and manage PyTorchTrainJobs programmatically)
* NFS-provisioner (used to enable logging on a persistent `ReadWriteMany` PVC in the cluster)


In [ ]:
terraform plan -out dependencies.out

In [ ]:
terraform apply -auto-approve

## Testing the deployment

To make sure that the deployment went OK, we can run the following command to test whether we can use Pytorch-Training operators.

This will create a simple deployment using a Kubeflow pytorch example job.

This will create a small (1 master, 1 client) training job on mnist on your cluster

In [ ]:
kubectl create -f https://raw.githubusercontent.com/kubeflow/training-operator/master/examples/pytorch/simple.yaml

# Cleaning up

To clean up/remove the cluster, or clean up the deploymen we have created, use the `terraform destroy` command.

This will remove everything defined in the Terraform configuration.

You can uncomment the commands below to remove the cluster, or run the command in a terminal in the [`../terraform/terraform-gke`](../terraform/terraform-gke) directory.

In [ ]:
# cd ../terraform-gke

# terraform destroy #-auto-approve